# Solve TSP using Cuckoo Search (CS) algorithm
The Cuckoo Search (CS) algorithm is a metaheuristic optimization algorithm developed by Xin-she Yang and Suash Deb in 2009. It is inspired by the brooding behavior of some cuckoo species.

To apply this algorithm to the Traveling Salesman Problem (TSP), we will define each "nest" as a sequence of cities (a path). The "quality" of a nest will be the total distance of the path; our goal is to minimize this distance.

Before we start, we'll need the following libraries:

- Numpy: For numerical computation
- Matplotlib: For visualizing our results
- Random: For generating random numbers

Here's a brief overview of how we'll approach this problem:

1. **Initialization**: Generate a population of random paths (nests)
2. **Cuckoo update**: Select a random nest and modify it (create a new solution)
3. **Quality assessment**: If the new solution is better, replace the nest with the new solution
4. **Eliminate worst nests**: Some fraction of the worst solutions are replaced with new, random solutions
5. **Iterate**: Repeat steps 2-4 until some stopping condition is met (e.g., maximum number of iterations, solution is good enough, etc.)

Now, let's proceed with the step by step code:

## Step 1: Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

## Step 2: Initialize Parameters

Before we start the optimization process, we need to set some parameters. In this case, we'll set the number of cities, the number of nests, the maximum number of iterations, and the fraction of nests we want to abandon.

In [2]:
# Parameters
num_cities = 50  # Number of cities
num_nests = 100  # Number of nests
max_iter = 1000  # Maximum number of iterations
pa = 0.25  # Fraction of worst nests to abandon

## Step 3: Generate Initial Population

Now we create our initial population of nests, which are simply random sequences of cities.

In [10]:
# Initialization
nests = [np.random.permutation(num_cities) for _ in range(num_nests)]

# Calculate distances
dist = np.zeros((num_cities, num_cities))
for i in range(num_cities):
    for j in range(i+1, num_cities):
        # Euclidean distance
        dist[i, j] = dist[j, i] = np.sqrt(np.sum((nests[i] - nests[j])**2))

print(nests)
print (dist)


[array([19, 32,  4, 16, 28, 20, 34, 13, 48, 33,  0, 15,  1, 44, 42, 36, 49,
        9, 29, 30, 14, 39, 22, 21,  2, 12, 37, 17, 38, 11, 24,  8,  7, 41,
        3, 10, 46,  6, 18, 43, 26, 45, 35, 40, 25, 27,  5, 47, 31, 23]), array([22, 21, 38, 29, 20, 39, 43, 17,  5,  9, 40, 35,  6, 46,  3, 23,  1,
       44, 31, 12,  7, 24, 16, 14, 30, 47, 19, 27, 10, 18, 41,  0, 37, 45,
       13, 28, 32,  8, 11, 48, 26, 34, 42,  2, 49,  4, 15, 33, 36, 25]), array([10,  1, 15, 33, 17, 29, 26, 43, 23, 14, 12, 45, 48, 28, 40, 39, 49,
       11, 46, 27, 16,  4, 41, 30, 44, 35,  9, 21, 37,  8, 22,  2, 47, 34,
        5,  0, 20,  6, 24, 36, 19,  3, 13, 25, 38, 32, 31, 18, 42,  7]), array([39, 29, 49, 37, 33, 45,  8, 23, 16, 13, 27, 31, 24, 14, 12, 43,  1,
       48, 18, 32,  9, 15, 21, 42, 17, 35, 46, 47, 36, 40, 25, 38, 41, 26,
        2, 28, 20,  3,  7, 22, 34,  0, 44, 30, 19,  6, 10, 11,  4,  5]), array([14, 24, 30, 15, 48, 46,  6, 37, 20, 26, 25, 16, 42, 44, 45,  3, 31,
        0,  8, 49, 12,  7, 23, 3

**Step 4: Define Helper Functions**

Next, we need some helper functions to compute the total distance of a path, generate a new solution from an existing one, and replace the worst nests.

In [8]:
# Function to calculate total distance
def path_dist(path):
    return sum(dist[path[i-1], path[i]] for i in range(num_cities))

# Function to generate a new solution (path)
def get_new_path(path):
    path = path.copy()
    i, j = random.sample(range(num_cities), 2)  # Pick two cities
    path[i], path[j] = path[j], path[i]  # Swap them
    return path

# Function to replace worst nests
def replace_worst_nests(nests, new_nests):
    # Calculate nest qualities
    nest_qualities = [path_dist(nest) for nest in nests]
    # Get indices of worst nests
    worst_nests = np.argsort(nest_qualities)[-int(pa*num_nests):]
    # Replace worst nests with new ones
    for i in worst_nests:
        nests[i] = new_nests[i]
    return nests

## Step 5: Main Loop

Finally, we can now run the main loop of our algorithm. At each iteration, we create new solutions for each nest, assess their quality, and replace the worst nests.

In [9]:
# Main loop
for _ in range(max_iter):
    # Generate new solutions
    new_nests = [get_new_path(nest) for nest in nests]
    # Replace nests if new solution is better
    for i in range(num_nests):
        if path_dist(new_nests[i]) < path_dist(nests[i]):
            nests[i] = new_nests[i]
    # Replace worst nests with new, random ones
    nests = replace_worst_nests(nests, [np.random.permutation(num_cities) for _ in range(num_nests)])

# Find the best path
best_path = min(nests, key=path_dist)

print("Best path:", best_path)
print("Total distance:", path_dist(best_path))


Best path: [ 4 32 49 43 31  3 28 33 37 21 10  5 14 46 25 42  1 39 12 23 36 19 47 18
 30 45 20 27  2 17 41 29  7 40 16  6  0 38 13 44 48 11  8 24 15 26  9 34
 35 22]
Total distance: 6386.011728829622
